In [1]:
# import sys
# sys.path.append('..')

import os

import numpy as np
import pandas as pd
import scipy as sp

# import tensorflow as tf
# from tensorflow.keras import Model, Input, losses, layers, optimizers

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score

from matplotlib import pyplot as plt
# from matplotlib.colors import BoundaryNorm
# import seaborn as sns

# import itertools
from tqdm.auto import trange, tqdm

from pathlib import Path

from src.datasets import adults
# from src.evaluation import data_benchmark
# from src.models import vae_keras
# from src.datasets import adults#, dataset_utils

# print(tf.config.list_physical_devices('GPU'))
# tf.debugging.set_log_device_placement(True)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# import tensorflow as tf
# for gpu in tf.config.list_physical_devices('GPU'):
#     tf.config.experimental.set_memory_growth(gpu, True)

ModuleNotFoundError: No module named 'tensorflow'

### This is just a scrappy notebook to temporarily reduce the dataset size of files in the ~\data\adult folder
#### (Restoring the original sizes requires copypasting from a backup of the original files)

In [90]:
y_t = np.load('../data/adult/y_train.npy', allow_pickle=True)
print(type(y_t))
print(np.unique(y_t, return_counts=True))

<class 'numpy.ndarray'>
(array([0, 1], dtype=int64), array([19775,  6273], dtype=int64))


In [91]:
import numpy as np
x_cat = np.load('../data/adult/X_cat_train.npy', allow_pickle=True)


# print(len(data))
# print(len(data[0]))
# print(data)
print(type(data))
print(data[:,-1])
print(len(data[:,-1]))
print((data[:,-1].shape))
print(np.sort(np.unique(data[:,-1])))
print(np.sort(np.unique(data[:,1])))
# y = data[:,-1]
# for i in range(len(y)):
#     print(y[i])
#     print(type(y[i]))
# print(type(np.unique(data[:,-1])[0]))

import numpy as np
data = np.load('../data/adult/X_num_train.npy', allow_pickle=True)

# print(len(data))
# print(len(data[0]))
# print(data)
print(type(data))
print(data[:,-1])
print(len(data[:,-1]))
print((data[:,-1].shape))
print(np.sort(np.unique(data[:,-1])))
print(np.sort(np.unique(data[:,1])))

<class 'numpy.ndarray'>
[30. 42.  4. ... 40. 40. 40.]
12546
(12546,)
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 70. 72. 73. 75.
 76. 77. 78. 80. 81. 82. 84. 85. 87. 88. 90. 96. 97. 98. 99.]
[  19214.   19302.   19520. ... 1226583. 1268339. 1455435.]
<class 'numpy.ndarray'>
[30. 42.  4. ... 40. 43. 40.]
26048
(26048,)
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 70. 72. 73. 75.
 76. 77. 78. 80. 81. 82. 84. 85. 86. 87. 88. 89. 90. 91. 92. 94. 95. 96.
 97. 98. 99.]
[  18827.   19214.   19302. ... 1268339. 1455435. 1484705.]


In [80]:
# Returns a Test dataset that contains an equal amounts of each class
# y should contain only two classes 0 and 1
def TrainSplitEqualBinary(X, y, samples_n): #samples_n per class
    
    indicesClass1 = []
    indicesClass2 = []
    
    for i in range(0, len(y)):
        if y[i] == 0 and len(indicesClass1) < samples_n:
            indicesClass1.append(i)
        elif y[i] == 1 and len(indicesClass2) < samples_n:
            indicesClass2.append(i)
            
        if len(indicesClass1) == samples_n and len(indicesClass2) == samples_n:
            break
    
    X_test_class1 = X[indicesClass1]
    X_test_class2 = X[indicesClass2]
    
    X_test = np.concatenate((X_test_class1,X_test_class2), axis=0)
    
    #remove x_test from X
    X_train = np.delete(X, indicesClass1 + indicesClass2, axis=0)
    
    Y_test_class1 = y[indicesClass1]
    Y_test_class2 = y[indicesClass2]
    
    y_test = np.concatenate((Y_test_class1,Y_test_class2), axis=0)
    
    #remove y_test from y
    y_train = np.delete(y, indicesClass1 + indicesClass2, axis=0)
    
    if (X_test.shape[0] != 2 * samples_n or y_test.shape[0] != 2 * samples_n):
        raise Exception("Problem with split 1!")
        
    if (X_train.shape[0] + X_test.shape[0] != X.shape[0] or y_train.shape[0] + y_test.shape[0] != y.shape[0]):
        raise Exception("Problem with split 2!")
    
    return X_train, X_test, y_train, y_test

In [81]:
for split in ['train']: #, 'test', 'val']:
    x_cat = np.load('../data/adult/X_cat_' + split +'.npy', allow_pickle=True) # 8 columns
    x_num = np.load('../data/adult/X_num_' + split +'.npy', allow_pickle=True) # 6 columns
    y = np.load('../data/adult/y_' + split+ '.npy', allow_pickle=True)
    
    print(x_cat.shape, x_num.shape, y.shape)
    
    x_train = np.concatenate([x_cat, x_num], axis=1)
    
#     x_train, x_test, y_train, y_test = train_test_split(x_train, y, train_size=1024, stratify=y)
    # 6273
    _, x_train, _, y_train = TrainSplitEqualBinary(x_train, y, 6273) 

    
    
    np.save('../data/adult/X_cat_' + split +'.npy', x_train[:,:8]) 
    np.save('../data/adult/X_num_' + split +'.npy', x_train[:,8:].astype(np.float32)) 
    np.save('../data/adult/y_' + split+ '.npy', y_train) 

    (print(x_train.shape))

    print(x_cat[:,-1])





(26048, 8) (26048, 6) (26048,)
(12546, 14)
['United-States' 'El-Salvador' 'United-States' ... 'United-States'
 'United-States' 'United-States']


In [83]:

print(x_cat[:,-1])

['United-States' 'El-Salvador' 'United-States' ... 'United-States'
 'United-States' 'United-States']


In [41]:
for split in ['train', 'test', 'val']:
    x_cat = np.load('X_cat_' + split +'.npy', allow_pickle=True) # 8 columns
    x_num = np.load('X_num_' + split +'.npy', allow_pickle=True) # 6 columns
    y = np.load('y_' + split+ '.npy', allow_pickle=True)
    
    x_train = np.concatenate([x_cat, x_num], axis=1)
    (print(x_train.shape))
    (print(y.shape))

(1024, 14)
(1024,)
(16281, 14)
(16281,)
(6513, 14)
(6513,)
